In [ ]:
def train_model(X_train, y_train, X_test, y_test):
    model = Sequential()

    model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    
    model.add(LSTM(64, return_sequences=False, kernel_regularizer=l1(0.001)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='relu', kernel_regularizer=l1(0.001)))
    model.add(Dense(1, activation='tanh'))  # tanh activation function for output range -1 to 1

    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    # Reduce learning rate when a metric has stopped improving
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
    callbacks = [reduce_lr]

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=400, batch_size=32, callbacks=callbacks)

    return model, history

# Fetch and preprocess data.
data = fetch_data()
pivot_points = calculate_pivot_points(data)
signals = generate_signals(data, pivot_points)

# Normalize data and create train and test datasets.
scaler = MinMaxScaler()
data[['High', 'Low', 'Close', 'Volume']] = scaler.fit_transform(data[['High', 'Low', 'Close', 'Volume']])
X = data[['High', 'Low', 'Close', 'Volume']]
y = signals['Signal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape input to be 3D [samples, timesteps, features]
time_steps = 1
X_train, y_train = create_dataset(X_train, y_train, time_steps)
X_test, y_test = create_dataset(X_test, y_test, time_steps)

# Train model.
model, history = train_model(X_train, y_train, X_test, y_test)
model.save('model.h5')
print("Model saved as model.h5.")


In [ ]:
# Extract the close prices.
close_prices = data['Close']

# Split close prices into training and test sets.
close_prices_train, close_prices_test = train_test_split(close_prices, test_size=0.2, random_state=42)

# Get rid of the first value of the test set to match the shape of the predictions.
close_prices_test = close_prices_test[1:]

# Create a simple backtest that counts the number of correct signals.
correct_signals = (np.sign(close_prices_test.values[1:] - close_prices_test.values[:-1]) == np.sign(predicted_signals[:-1, 0])).sum()
total_signals = len(close_prices_test) - 1

# Print the accuracy of the model.
print(f"Signal accuracy: {correct_signals / total_signals:.2f}")

# Let's simulate the investment value if we used this model
investment = 10000.0  # initial investment
n_shares = 0  # number of shares we own
for pred, close_today, close_tomorrow in zip(predicted_signals, close_prices_test.values[:-1], close_prices_test.values[1:]):
    if pred[0] > 0:
        # Buy shares only if we have enough money and the model predicts the price will go up
        n_new_shares = investment // close_today
        n_shares += n_new_shares
        investment -= n_new_shares * close_today
    elif pred[0] < 0 and n_shares > 0:
        # Sell all shares if the model predicts the price will go down
        investment += n_shares * close_today
        n_shares = 0
# Calculate the final value of our investment (investment money + value of shares we own)
investment += n_shares * close_prices_test.values[-1]

print(f"Final investment value: {investment:.2f}")


In [ ]:
data.head()

In [ ]:
import matplotlib.pyplot as plt

# Initialize variables
cash = 10000.0  # Starting cash
stock = 0.0  # Starting stock units
investment_values = []  # To track the value of our investment over time

# We simulate buying and selling based on the predicted signals.
# We assume that we buy or sell one stock unit whenever the signal is positive or negative.
for i in range(len(predicted_signals)):
    # We calculate the total investment value (stock value + cash) and append it to investment_values.
    investment_values.append(stock * close_prices_test.values[i] + cash)
    if predicted_signals[i] > 0 and cash >= close_prices_test.values[i]:
        # Buy signal: we buy one stock unit using the closing price of the day.
        stock += 1.0
        cash -= close_prices_test.values[i]
    elif predicted_signals[i] < 0 and stock > 0:
        # Sell signal: we sell one stock unit.
        stock -= 1.0
        cash += close_prices_test.values[i]

# Adding the final day's investment value
investment_values.append(stock * close_prices_test.values[-1] + cash)
    
# Now we convert investment_values to a time series for easier plotting.
investment_values = pd.Series(investment_values, index=close_prices_test.index)

# Plot the investment value over time
plt.figure(figsize=(10, 5))
plt.plot(investment_values)
plt.xlabel('Time')
plt.ylabel('Investment Value')
plt.title('Investment Value Over Time')
plt.grid(True)
plt.show()
